# Okamoto-Tanaka Revisited protocol


In [ ]:
import random
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import math

KEY_SIZE = 512
H1_EXPONENT_SIZE = 256
SESSION_KEY_SIZE = 128
EXPONENT_SIZE = 256
ID_SIZE = 256
ENDIAN = "little"


In [ ]:
#dobór funkcji haschującej
def select_SHA_context(length_in_bits):
    if length_in_bits <= 224:
        return hashes.SHA224()
    elif length_in_bits <= 256:
        return hashes.SHA256()
    elif length_in_bits <= 384:
        return hashes.SHA384()
    elif length_in_bits <= 512:
        return hashes.SHA512()
    else:
        #TODO Exception
        print("Error, output to big for now")

#hf_KGC = select_SHA_context(H1_EXPONENT_SIZE)
#hf_USER = select_SHA_context(SESSION_KEY_SIZE)


In [ ]:
#Użytkownik
class User:
    def __init__(self,name,user_id):
        self.name = name
        self.id = user_id.to_bytes(ID_SIZE//8, byteorder=ENDIAN)
        
    def set_private_key(self,key):
        self.private_key = key
    
user_A = User("A",random.randint(1,2**ID_SIZE))
user_B = User("B",random.randint(1,2**ID_SIZE))

## Generacja klucza
Generujemy klucze uży


### Generacja klucza RSA:

In [ ]:
class KGC_key_pair:
    def __init__(self):
        rsa_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size = KEY_SIZE,
            backend = default_backend()
        )
        self.N = rsa_key.public_key().public_numbers().n
        self.e = rsa_key.public_key().public_numbers().e
        self.d = rsa_key.private_numbers().d
        p = rsa_key.private_numbers().p
        q = rsa_key.private_numbers().q
        self.Phi = (p-1)*(q-1)
    def set_QRN_generator(self, generator):
        self.QRN_generator = generator
kgc01 = KGC_key_pair()
        

In [ ]:
#rsa_key = rsa.generate_private_key(
#    public_exponent=65537,
#    key_size = KEY_SIZE,
#    backend=default_backend()
#)
#N = rsa_key.public_key().public_numbers().n
#p = rsa_key.private_numbers().p
#q = rsa_key.private_numbers().q
#Phi = (p-1)*(q-1)
#print("N=",N)
#print("p=",p)
#print("q=",q)
#if Phi % 2 != 0:
#    print ("Phi should be even!")
#print("Phi=",Phi)




### Generacja kluczy mOT

### Znalezienie generatora g grupy reszt kwadratowych mod N:

#### Rozkład N-1 = d * (2^r)

In [ ]:
#Rozłożenie N-1 = d* (2**r)
def mr(number):
    t = number
    r = 0
    while t%2 == 0:
    #while math.fmod(d,2) ==0:
        t//=2
        r+=1
    return (t,r)

(t,r) = mr(kgc01.Phi)
print(t,r)
print(t*(2**r))


#### Wyznaczenie generatora g grupy QR_N

In [ ]:
#generator grupy Z_N: Znaleźć alpha że g = alpha**2 jest generatorem dużej podgrupy (reszt kwadratowych)
cont = True

while cont == True:
    alpha = random.randint(1,kgc01.N)
    #print (pow(alpha, Phi, N))
    beta = pow(alpha,t,kgc01.N)
    print (beta, alpha)
    if  beta != 1:
        cont = False
        #print ("alpha jest generatorem dużej podgrupy")
    
g = pow(alpha,2,kgc01.N)
kgc01.set_QRN_generator(g)

In [ ]:
# funkcja hashująca do grupy reszt kwadratowych
def hf_KGC(kgc, material, length_in_bits = H1_EXPONENT_SIZE, key_size = KEY_SIZE, endian = ENDIAN):
    context = hashes.Hash(select_SHA_context(length_in_bits), backend=default_backend()) 
    context.update(material)
    exponent = int.from_bytes(context.finalize(), byteorder = endian)
    sk = pow(kgc.QRN_generator, exponent, kgc01.N)
    return pow(sk, kgc.d, kgc.N).to_bytes(key_size//8, endian)

sk = hf_KGC(kgc01, user_A.id)
print(sk)
    

    

#### Generacja klucza


In [ ]:


def generate_user_key(user_id, hash_function, private_exponent):
    return 0

#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="little"))
#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="big"))
print(user_A.id)



#print(idA)
#print(hex(user_A.id))

In [ ]:
#TEST
digest1 = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest2 = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest3 = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest1.update(b"abc")
digest1.update(b"123")
digest2.update(b"123")
digest3.update(b"abc123")

print(digest1.finalize())
print(digest2.finalize())
print(digest3.finalize())

print("1:\t", digest1)
print("2:\t", digest2)
print("3:\t", digest3)
    #TODO

#Generacja klucza użytkownika A:

#user_A.set_private_key(pk)
#Generacja klucza użytkownika B:

#user_B.set_private_key(pk)